In [1]:
import polars
import numpy
df = polars.read_csv("descriptor_selfies.csv", separator='\t')
df.head()

id,label,one_hot
str,str,str
"""CHEMBL179549""","""[24, 24, 24, 30, 24, 7, 24, 12…","""[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0…"
"""CHEMBL360920""","""[24, 24, 24, 30, 24, 7, 24, 12…","""[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0…"
"""CHEMBL182052""","""[24, 24, 24, 33, 24, 17, 8, 24…","""[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0…"
"""CHEMBL179662""","""[33, 24, 24, 18, 36, 24, 30, 2…","""[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0…"
"""CHEMBL181688""","""[24, 24, 17, 24, 24, 17, 24, 2…","""[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0…"


In [2]:
X, y= df["one_hot"], df["label"]

In [3]:
y_num = []
for y_str in y:
    y_list = eval(y_str)
    y_num.append(y_list)
y = numpy.array(y_num) # create multi-outputs labels
del y_str, y_list, y_num

In [4]:
X_num = []
for X_str in X:
    X_list = eval(X_str)
    X_flatten = [item for sublist in X_list for item in sublist]
    X_num.append(X_flatten)
X = numpy.array(X_num) # create flatten features
del X_str, X_list, X_flatten, X_num

In [5]:
X.shape, y.shape

((1325, 8862), (1325, 211))

In [6]:
import perming
main = perming.Box(8862, 211, (3000,), batch_size=64, activation='relu', inplace_on=True, criterion='MSELoss', solver='adam', learning_rate_init=0.01)
main.print_config() # extract features with main effects

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=8862, out_features=3000, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=3000, out_features=211, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MSELoss()),
             ('batch_size', 64),
             ('solver',
              Adam (
              Parameter Group 0
                  amsgrad: False
                  betas: (0.9, 0.99)
                  eps: 1e-08
                  lr: 0.01
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

In [7]:
y = y.astype("float") # multi-outputs
main.data_loader(X, y, random_seed=0)

In [8]:
main.train_val(num_epochs=60, interval=10, backend='threading', prefer='threads', early_stop=True) # set n_jobs > 1 within number of processes

Epoch [1/60], Step [10/17], Training Loss: 683.9639, Validation Loss: 158.5208
Epoch [2/60], Step [10/17], Training Loss: 74.4517, Validation Loss: 16.8641
Epoch [3/60], Step [10/17], Training Loss: 41.3552, Validation Loss: 13.4700
Epoch [4/60], Step [10/17], Training Loss: 39.0044, Validation Loss: 13.0534
Epoch [5/60], Step [10/17], Training Loss: 35.2509, Validation Loss: 11.4001
Epoch [6/60], Step [10/17], Training Loss: 31.8014, Validation Loss: 10.8461
Epoch [7/60], Step [10/17], Training Loss: 29.4561, Validation Loss: 9.9555
Epoch [8/60], Step [10/17], Training Loss: 29.6944, Validation Loss: 8.6690
Epoch [9/60], Step [10/17], Training Loss: 26.8422, Validation Loss: 9.0475
Epoch [10/60], Step [10/17], Training Loss: 23.5534, Validation Loss: 8.4559
Epoch [11/60], Step [10/17], Training Loss: 22.1152, Validation Loss: 7.9635
Epoch [12/60], Step [10/17], Training Loss: 20.2078, Validation Loss: 7.2285
Epoch [13/60], Step [10/17], Training Loss: 18.4414, Validation Loss: 6.6722


In [9]:
main.test()

loss of Box on the 192 test dataset: 5.462291717529297.


OrderedDict([('problem', 'multi-outputs'),
             ('loss',
              {'train': 1.0389052629470825,
               'val': 1.822463035583496,
               'test': 5.462291717529297})])

In [10]:
main.save(con=False, dir='./models/tp_selfies.ckpt')